In [1]:
from slim_gsgp_lib_np.main_slim import slim
from slim_gsgp_lib_np.utils.utils import train_test_split
from slim_gsgp_lib_np.utils.callbacks import *
from slim_gsgp_lib_np.evaluators.fitness_functions import rmse
from matplotlib.colors import LinearSegmentedColormap
import numpy as np
import time
import os
from tqdm import tqdm
from functions.test_funcs import mape, nrmse, r_squared, mae, standardized_rmse
from matplotlib import pyplot as plt
from slim_gsgp_lib_np.algorithms.SLIM_GSGP.operators.mutators import *
from slim_gsgp_lib_np.algorithms.SLIM_GSGP.operators.simplifiers import *
from slim_gsgp_lib_np.datasets.data_loader import *
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import pdist, squareform

datasets = [globals()[i] for i in globals() if 'load' in i][2:]

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["BLIS_NUM_THREADS"] = "1"


# -------------------------- # 
from slim_gsgp_lib_np.utils.utils import check_slim_version
from slim_gsgp_lib_np.algorithms.MULTI_SLIM.representations.tree import Tree
from slim_gsgp_lib_np.algorithms.MULTI_SLIM.representations.population import Population    
from slim_gsgp_lib_np.algorithms.MULTI_SLIM.representations.tree_utils import *

In [2]:
# X, y = datasets[0]()
# X_train, X_test, y_train, y_test = train_test_split(X, y, p_test=0.2)
# X_train = MinMaxScaler().fit_transform(X_train)
# X_test = MinMaxScaler().fit_transform(X_test)
# y_train = MinMaxScaler().fit_transform(y_train.reshape(-1, 1)).ravel()
# y_test = MinMaxScaler().fit_transform(y_test.reshape(-1, 1)).ravel()

X = np.random.rand(20, 2)
y = np.random.rand(20)

X_train, X_test, y_train, y_test = train_test_split(X, y, p_test=0.2)

seed = 2

# agelog = LogAge()
# divlog = LogDiversity()
# early_stop = EarlyStopping_train(patience=2500)

example_tree, population = slim(X_train=X_train, y_train=y_train,
                    dataset_name='test', test_elite=False, slim_version='SLIM*ABS', # initializer='simple',
                    max_depth=14, init_depth=6, pop_size=100, n_iter=100, seed=seed, verbose=1,
                    p_inflate=0.3, p_struct=0.2, eps_fraction=1e-5,
                    prob_const=0.1, n_elites=1, selector='e_lexicase', 
                    decay_rate=0.2, p_xo=0, p_struct_xo=0, prob_terminal=0.8,
                    # callbacks=[agelog, divlog, early_stop], 
                    mode='exp', tournament_size=2, full_return=True, timeout=200,
    )

preds = example_tree.predict(X_test)
print('RMSE:', rmse(preds, y_test))

+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|     dataset     |        it       |      train      |       test      |       time      |      nodes      |       div       |     avgStru     |      avgDep     |      struct     |     inflate     |     deflate     |        xo       |
+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|       test      |        0        |      0.184      |       None      |      0.044      |        5        |        23       |      3.150      |      3.150      |     N/A (0)     |     N/A (0)     |     N/A (0)     |     N/A (0)     |
|-----------------|-----------------|-----------------|-

In [3]:
for ind in population.population:
    ind.version = example_tree.version
    ind.train_semantics = ind.predict(X_train)
    ind.test_semantics = ind.predict(X_test)

In [4]:
FUNCTIONS = example_tree.collection[0].FUNCTIONS
TERMINALS = example_tree.collection[0].TERMINALS
CONSTANTS = example_tree.collection[0].CONSTANTS
SPECIALISTS = {f'S_{i}' : ind for i, ind in enumerate(population.population) if i<5}
Tree.FUNCTIONS = FUNCTIONS
Tree.TERMINALS = TERMINALS
Tree.CONSTANTS = CONSTANTS
Tree.SPECIALISTS = SPECIALISTS

In [5]:
pop = initializer(init_pop_size=100, 
            depth_condition=3, 
            depth_tree=3, 
            FUNCTIONS=FUNCTIONS, 
            TERMINALS=TERMINALS, 
            CONSTANTS=CONSTANTS,
            SPECIALISTS=SPECIALISTS,
            p_c=0.3, 
            p_t=0.7, 
            p_specialist=0.7,
            )

In [6]:
population = Population(pop=[Tree(struc) for struc in pop])

In [7]:
population.calculate_semantics(X_train, testing=False)

In [8]:
population.evaluate(y_train, testing=False)

In [364]:
struc = create_random_tree(3, 3, FUNCTIONS, TERMINALS, CONSTANTS, SPECIALISTS, p_specialist=0.7, p_t=0.7)
print(struc)

(('divide', 'x0', ('subtract', 'x1', 'x0')), 'S_0', 'S_2')


In [9]:
struc = create_random_tree(2, 1, FUNCTIONS, TERMINALS, CONSTANTS, SPECIALISTS)
tree = Tree(struc)
tree.evaluate(rmse, X_train, y_train, testing=False)
tree.evaluate(rmse, X_test, y_test, testing=True)   
tree.print_tree_representation()
print('------------------/-----------------')
print('Train RMSE (ensemble):', tree.fitness)
print('Test RMSE (ensemble):', tree.test_fitness)
print('Train RMSE (individual):', rmse(example_tree.predict(X_train), y_train))
print('Test RMSE (individual):', rmse(example_tree.predict(X_test), y_test))
print('------------------/-----------------')
print('Nodes:', tree.nodes_count)   
print('Total Nodes:', tree.total_nodes)


if (
  multiply(
    constant_0.41
    x0
  )
) > 0 then
  S_3
else
  S_2
endif

------------------/-----------------
Train RMSE (ensemble): 0.5768332842641796
Test RMSE (ensemble): 0.6984042546297496
Train RMSE (individual): 0.14272031425948212
Test RMSE (individual): 0.27550132633252133
------------------/-----------------
Nodes: 5
Total Nodes: 103
